In [2]:
import requests
import pandas as pd

# here you have to enter your actual API key from SimFin
api_key = "b8vabmV9ZlSP7TCmUYewGrRJAyS8Jc8V"

# list of tickers we want to get data for
tickers = ["AAPL", "GOOG", "WMT"]

# first: find SimFin IDs
sim_ids = []
for ticker in tickers:

    request_url = f'https://simfin.com/api/v1/info/find-id/ticker/{ticker}?api-key={api_key}'
    content = requests.get(request_url)
    data = content.json()

    if "error" in data or len(data) < 1:
        sim_ids.append(None)
    else:
        sim_ids.append(data[0]['simId'])

print(sim_ids)

# define time periods for financial statement data
statement_type = "pl"
time_periods = ["Q1", "Q2", "Q3", "Q4"]
year_start = 2013
year_end = 2018

# set up the XLSX writer
writer = pd.ExcelWriter("simfin_data.xlsx", engine='xlsxwriter')

list_of_df = []

data = {}
# get data for each ticker/id
for idx, sim_id in enumerate(sim_ids):
    d = data[tickers[idx]] = {"Line Item": []}
    if sim_id is not None:
        for year in range(year_start, year_end + 1):
            for time_period in time_periods:

                # make time period identifier
                period_identifier = time_period + "-" + str(year)

                if period_identifier not in d:
                    d[period_identifier] = []

                request_url = f'https://simfin.com/api/v1/companies/id/{sim_id}/statements/standardised?stype={statement_type}&fyear={year}&ptype={time_period}&api-key={api_key}'

                content = requests.get(request_url)
                statement_data = content.json()

                # collect line item names once, they are the same for all companies with the standardised data
                if len(d['Line Item']) == 0 and 'values' in statement_data:
                    d['Line Item'] = [x['standardisedName'] for x in statement_data['values']]

                if 'values' in statement_data:
                    for item in statement_data['values']:
                        d[period_identifier].append(item['valueChosen'])
                else:
                    # no data found for time period
                    d[period_identifier] = [None for _ in d['Line Item']]

        # fix the periods where no values were available
        len_target = len(d['Line Item'])
        if len_target > 0:
            for k, v in d.items():
                if len(v) != len_target:
                    d[k] = [None for _ in d['Line Item']]
                    
    # convert to pandas dataframe
    df = pd.DataFrame(data=d)
    df = df.transpose()
    df['ticker'] = tickers[idx]
    list_of_df += [df]
    # save in the XLSX file configured earlier
    df.to_excel(writer, sheet_name=tickers[idx])

# writer.save()
# writer.close()

[111052, 18, 239962]


In [4]:
final_df = pd.concat(list_of_df)

In [5]:
final_df[final_df.ticker == 'GOOG']

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,ticker
Line Item,Revenue,Sales & Services Revenue,Financing Revenue,Other Revenue,Cost of revenue,Cost of Goods & Services,Cost of Financing Revenue,Cost of Other Revenue,Gross Profit,Other Operating Income,...,Net Extraordinary Gains (Losses),Discontinued Operations,XO & Accounting Charges & Other,Income (Loss) Including Minority Interest,Minority Interest,Net Income,Preferred Dividends,Other Adjustments,Net Income Available to Common Shareholders,GOOG
Q1-2013,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,GOOG
Q2-2013,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,GOOG
Q3-2013,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,GOOG
Q4-2013,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,GOOG
Q1-2014,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,GOOG
Q2-2014,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,GOOG
Q3-2014,16523000000,None,None,None,-6695000000,None,None,None,9828000000,None,...,-185000000,-185000000,None,2739000000,None,2739000000,None,0,2739000000,GOOG
Q4-2014,1.8103e+10,NaN,NaN,NaN,-6.921e+09,NaN,NaN,NaN,1.1182e+10,NaN,...,9.67e+08,9.67e+08,NaN,4.675e+09,NaN,4.675e+09,NaN,0,4.675e+09,GOOG
Q1-2015,17258000000,None,None,None,-6356000000,None,None,None,10902000000,None,...,None,None,None,3515000000,None,3515000000,None,None,3515000000,GOOG
